## Threading

Readings:
- https://docs.python.org/es/3/library/threading.html
- https://www.educative.io/edpresso/what-are-locks-in-python
- https://www.bogotobogo.com/python/Multithread/python_multithreading_Using_Locks_with_statement_Context_Manager.php

## Basic methods to get info about the threads

In [1]:
import random
import time
import threading
from threading import excepthook as original_excepthook


def print_thread(*args, **kwargs):
    current_thread = threading.current_thread()
    print(f"thread: {current_thread} - {threading.active_count()}\n")
    print(f"dir:\n{dir(current_thread)}\n")
    print(f"dict:\n{current_thread.__dict__}\n")
    if current_thread._name == 'CustomName':
        raise Exception

        
def excepthook(arg):
    print('excepthook:')
    print(f'arg: {arg}')

    
class MyThread(threading.Thread):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        
frame = None
def func(*args):
    global frame
    frame = args[0]

    
def profile(frame, event, arg):
    # print(f'{frame!r} {event!r} {arg!r}')
    pass


threading.settrace(func)
threading.setprofile(profile)
threading.excepthook = excepthook

print(f'{" without thread ":=^50}')
print_thread(1, number=1)

print(f'{" with thread ":=^50}')

thread = MyThread(name='CustomName', target=print_thread, args=(1,), kwargs={'number': 2})
thread.start()
thread.join()

print(f'\n{" list of threads ":=^50}')
print(threading.enumerate())

print(f'\n{" frame ":=^50}')
print(frame)
print(dir(frame))

threading.excepthook = original_excepthook

================= without thread =================
thread: <_MainThread(MainThread, started 140331379058496)> - 5

dir:
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_args', '_bootstrap', '_bootstrap_inner', '_daemonic', '_delete', '_ident', '_initialized', '_invoke_excepthook', '_is_stopped', '_kwargs', '_name', '_native_id', '_reset_internal_locks', '_set_ident', '_set_native_id', '_set_tstate_lock', '_started', '_stderr', '_stop', '_target', '_tstate_lock', '_wait_for_tstate_lock', 'daemon', 'getName', 'ident', 'isAlive', 'isDaemon', 'is_alive', 'join', 'name', 'native_id', 'run', 'setDaemon', 'setName', 'start']

dict:
{'_target': None, '_name': 'MainThread', '_args': (), '_kwargs': {}, 

## Exercise with requests sessions

In [2]:
import requests


def get_session(local_thread):
    if not hasattr(local_thread, 'session'):
        thread.session = requests.Session()
    return thread.session


def get_site(site, local_thread):
    session = get_session(local_thread)
    with session.get(site) as response:
        print(site)


local_thread = threading.local()
for site in ('https://google.com', 'https://youtube.com', 'https://gmail.com', 'https://translate.google.com/'):
    threading.Thread(target=get_site, args=(site,local_thread)).start()

https://translate.google.com/
https://google.com
https://youtube.com
https://gmail.com


## Locks

### Thread without lock

In [3]:
class Customer:
    def __init__(self, money):
        self.money = money
    def reduce(self, amount):
        self.money -= amount
    def increment(self, amount):
        self.money += amount

        
def repeat_function(times, function, args):
    for _ in range(times):
        function(args)

        
customer = Customer(100)


thread_1 = threading.Thread(target=repeat_function, args=(1000000, customer.reduce, 100))
thread_2 = threading.Thread(target=repeat_function, args=(1000000, customer.increment, 100))
thread_1.start()
thread_2.start()
thread_1.join()
thread_2.join()
print(customer.money)

5439800


### Thread with lock

Same lock for differents tasks

In [4]:
class Customer:
    def __init__(self, money):
        self.money = money
        self.lock = threading.Lock()
    def reduce(self, amount):
        with self.lock:
            self.money -= amount
    def increment(self, amount):
        with self.lock:
            self.money += amount

            
def repeat_function(times, function, args):
    for _ in range(times):
        function(args)

        
customer = Customer(100)
thread_1 = threading.Thread(target=repeat_function, args=(1000000, customer.reduce, 100))
thread_2 = threading.Thread(target=repeat_function, args=(1000000, customer.increment, 100))
thread_1.start()
thread_2.start()
thread_1.join()
thread_2.join()
print(customer.money)

100


Different locks for differents tasks

In [5]:
class Customer:
    def __init__(self, money):
        self.money = money
        self.lock_reduce = threading.Lock()
        self.lock_increment = threading.Lock()
    def reduce(self, amount):
        with self.lock_reduce:
            self.money -= amount
    def increment(self, amount):
        with self.lock_increment:
            self.money += amount

            
def repeat_function(times, function, args):
    for _ in range(times):
        function(args)

        
customer = Customer(100)
thread_1 = threading.Thread(target=repeat_function, args=(1000000, customer.reduce, 100))
thread_2 = threading.Thread(target=repeat_function, args=(1000000, customer.increment, 100))
thread_1.start()
thread_2.start()
thread_1.join()
thread_2.join()
print(customer.money)

2227300


With lock but without custom thread

In [6]:
class Customer:

    def __init__(self, money):
        self.money = money
        self.lock = threading.Lock()

    def reduce(self, amount):
        with self.lock:
            self.money -= amount

    def increment(self, amount):
        with self.lock:
            self.money += amount

            
def repeat_function(times, function, args):
    for _ in range(times):
        function(args)

        
customer = Customer(100)
repeat_function(1000000, customer.reduce, 100)
repeat_function(1000000, customer.increment, 100)
print(customer.money)

100


### Recursion using RLock

In [7]:
class Factorial:
    value = 1
    
    def __init__(self):
        self.lock = threading.RLock()
    
    def set_factorial(self, n):
        with self.lock:
            if n > 1:
                self.value *= n
                self.set_factorial(n - 1)


factorial = Factorial()
thread = threading.Thread(target=factorial.set_factorial, args=(5,))
thread.start()
thread.join()
print(factorial.value)

120


## Conditions

Solving https://es.stackoverflow.com/questions/404798/python-threading-condition

In [8]:
class Cocina:

    def __init__(self):
        self.condition = threading.Condition()
        self.vaso_ocupado = True

    def __beber_del_vaso(self):
        """
        Ponemos el vaso como ocupado y simulamos que
        se está bebiendo de él.
        Cuando se termina, notificamos al siguiente hilo
        en la cola.
        """
        self.vaso_ocupado = True
        self.print('comenzó a beber')
        time.sleep(int(random.randint(1, 3)))  # reduce time from 10 to 3
        self.print('terminó de beber')
        self.vaso_ocupado = False
        self.condition.notify()
        self.condition.release()

    @staticmethod
    def print(msg):
        print(f'{threading.current_thread().name} - {msg}')

    def esperando_el_vaso(self):
        """
        Se unen las instancias a un estado de espera,
        donde, cuando sean notificadas, recién podrán
        beber del vaso.
        """
        self.condition.acquire()
        while self.vaso_ocupado:
            self.print('esperando el vaso')
            self.condition.wait()
        self.__beber_del_vaso()

    def habilitar_vaso(self):
        """
        Notificamos a los hilos que se encuentran en espera,
        de que el vaso ya está disponible.
        """
        self.print('vaso habilitado')
        self.condition.acquire()
        self.vaso_ocupado = False
        self.condition.notify()
        self.condition.release()

        
cocina = Cocina()
threads = []
for _ in range(5):  # reduce threads from 20 to 5
    thread = threading.Thread(target=cocina.esperando_el_vaso)
    thread.start()
    threads.append(thread)
time.sleep(5)  # wait some seconds before making the `vaso` available
cocina.habilitar_vaso()

Thread-15 - esperando el vaso
Thread-16 - esperando el vaso
Thread-17 - esperando el vaso
Thread-18 - esperando el vaso
Thread-19 - esperando el vaso
MainThread - vaso habilitado
Thread-15 - comenzó a beber
Thread-15 - terminó de beber
Thread-16 - comenzó a beber
Thread-16 - terminó de beber
Thread-17 - comenzó a beber
Thread-17 - terminó de beber
Thread-18 - comenzó a beber
Thread-18 - terminó de beber
Thread-19 - comenzó a beber
Thread-19 - terminó de beber


## Semaphore

In [9]:
class Restaurant:

    def __init__(self, amount_of_chairs):
        self.chairs_available = threading.BoundedSemaphore(amount_of_chairs)

    def can_reserve(self, amount_of_chairs):
        return self.chairs_available._value - amount_of_chairs >= 0

    def reserve(self, amount_of_chairs):
        if self.can_reserve(amount_of_chairs):
            for _ in range(amount_of_chairs):
                self.chairs_available.acquire()
            return True
        return False

    def cancel_reserve(self, amount_of_chairs):
        for _ in range(amount_of_chairs):
            self.chairs_available.release()


Tres56 = Restaurant(50)
Tres56.reserve(50)
print(Tres56.can_reserve(1))
print(Tres56.reserve(1))

Tres56.cancel_reserve(50)
try:
    Tres56.cancel_reserve(1)
except ValueError:
    print('Exception caught')

False
False
Exception caught


## Events

In [10]:
class AvenueSemaphore:

    def __init__(self):
        self.semaphore = threading.Event()

    def transit_the_avenue(self):
        if not self.semaphore.is_set():
            print(f'{threading.current_thread().name} - waiting for the semaphore')
            self.semaphore.wait()
        print(f'{threading.current_thread().name} - proceed over the avenue')

        
BvGLehmann = AvenueSemaphore()

BvGLehmann.semaphore.clear()

for identification in ('A', 'B', 'C'):
    time.sleep(0.1)
    threading.Thread(name=identification, target=BvGLehmann.transit_the_avenue).start()

time.sleep(1)

BvGLehmann.semaphore.set()
for identification in ('D', 'E', 'F'):
    time.sleep(0.1)
    threading.Thread(name=identification, target=BvGLehmann.transit_the_avenue).start()

A - waiting for the semaphore
B - waiting for the semaphore
C - waiting for the semaphore
B - proceed over the avenue
C - proceed over the avenue
A - proceed over the avenue
D - proceed over the avenue
E - proceed over the avenue
F - proceed over the avenue


## Timer

In [11]:
def message():
    print('message')

timer = threading.Timer(1, message)
timer.start()
time.sleep(2)
timer.cancel()

timer = threading.Timer(2, message)
timer.start()
time.sleep(1)
timer.cancel()

message


## Barrier

In [12]:
def message():
    print('March!')

    
def in_position(barrier):
    print(f'{threading.current_thread().name} in position!')
    barrier.wait()
    
    
militaries = 10
formation = threading.Barrier(militaries, message)
for n in range(militaries):
    time.sleep(0.1)
    threading.Thread(name=f'Soldier #{n + 1}', target=in_position, args=(formation,)).start()

Soldier #1 in position!
Soldier #2 in position!
Soldier #3 in position!
Soldier #4 in position!
Soldier #5 in position!
Soldier #6 in position!
Soldier #7 in position!
Soldier #8 in position!
Soldier #9 in position!
Soldier #10 in position!
March!


## Motorcyclist problem

A front-end problem made with python.

The Motorcycle model acts as a backend, where it gives the validations to the User model.

Related link: https://youtu.be/aouDQ8caJYg?t=225

In [14]:
import collections
import datetime
import logging
import queue
import threading
import time

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler()
handler.setFormatter(logging.Formatter('[%(threadName)s]: %(message)s'))
logger.addHandler(handler)

AVAILABLE = 0
CONTRACTED = 1
EMPTY = 2


class MotorcyclistsGroup:

    def __init__(self, amount, minutes):
        date = datetime.datetime(1, 1, 1) + datetime.timedelta(minutes=minutes)
        self.hour = date.hour
        self.minutes = date.minute

        self.availables = queue.Queue(amount)
        for n in range(amount):
            self.availables.put(f'Motorcyclist #{n} ({self})')

        self.users = {}
        self.users_lock = threading.Lock()

    def __repr__(self):
        return str(self)

    def __str__(self):
        return f'{self.hour:0>2}:{self.minutes:0>2}'
        
    def is_empty(self):
        return self.availables.empty()

    def count(self):
        return self.availables.qsize()

    def toggle(self):
        thread_name = threading.current_thread().name
        with self.users_lock:
            status = self.check_status()
            if status == CONTRACTED:
                data = self.users.pop(thread_name)
                self.availables.put(data)
                return f'Released {data}'
            elif status == AVAILABLE:
                data = self.availables.get()
                self.users[thread_name] = data
                return f'Get {data}'
            return 'Invalid action'

    def check_status(self):
        if threading.current_thread().name in self.users:
            return CONTRACTED
        elif self.is_empty():
            return EMPTY
        return AVAILABLE


class User:

    def __init__(self):
        self.contracted_schedules = collections.defaultdict(lambda: None)

    def click(self, schedule):
        data = schedule.toggle()
        self.contracted_schedules[schedule] = data
        return data


def check(schedule):
    status = {
        AVAILABLE: 'White',
        CONTRACTED: 'Green',
        EMPTY: 'Red',
    }
    logger.info('Status (%(schedule)s): %(status)s', {'schedule': schedule, 'status': status[schedule.check_status()]})
    logger.info('Amount (%(schedule)s): %(count)s' , {'schedule': schedule, 'count': schedule.count()})


def user_1_actions(motorcyclists):
    user = User()

    check(motorcyclists[0])
    check(motorcyclists[1])

    logger.info(user.click(motorcyclists[0]))
    check(motorcyclists[0])
    check(motorcyclists[1])

    time.sleep(4)

    logger.info(user.click(motorcyclists[1]))
    check(motorcyclists[0])
    check(motorcyclists[1])
    
    logger.info(user.click(motorcyclists[0]))
    check(motorcyclists[0])
    check(motorcyclists[1])


def user_2_actions(motorcyclists):
    user = User()

    time.sleep(1)

    check(motorcyclists[0])
    check(motorcyclists[1])

    logger.info(user.click(motorcyclists[0]))
    check(motorcyclists[0])
    check(motorcyclists[1])

    logger.info(user.click(motorcyclists[1]))
    check(motorcyclists[0])
    check(motorcyclists[1])


def user_3_actions(motorcyclists):
    user = User()

    time.sleep(2)

    check(motorcyclists[0])
    check(motorcyclists[1])
    
    logger.info(user.click(motorcyclists[0]))
    check(motorcyclists[0])
    check(motorcyclists[1])


def user_4_actions(motorcyclists):
    user = User()

    time.sleep(3)

    check(motorcyclists[0])
    check(motorcyclists[1])


motorciclists_amount = 3  # Default 8
intervals = 2  # Default 25
motorcyclists = [MotorcyclistsGroup(2, 480 + 30 * n) for n in range(intervals)]
logger.info(motorcyclists)
user_1 = threading.Thread(name='User 1', target=user_1_actions, args=(motorcyclists,))
user_2 = threading.Thread(name='User 2', target=user_2_actions, args=(motorcyclists,))
user_3 = threading.Thread(name='User 3', target=user_3_actions, args=(motorcyclists,))
user_4 = threading.Thread(name='User 4', target=user_4_actions, args=(motorcyclists,))
users = [user_1, user_2, user_3, user_4]
for user in users:
    user.start()
for user in users:
    user.join()

[MainThread]: [08:00, 08:30]
[MainThread]: [08:00, 08:30]
[User 1]: Status (08:00): White
[User 1]: Status (08:00): White
[User 1]: Amount (08:00): 2
[User 1]: Amount (08:00): 2
[User 1]: Status (08:30): White
[User 1]: Status (08:30): White
[User 1]: Amount (08:30): 2
[User 1]: Amount (08:30): 2
[User 1]: Get Motorcyclist #0 (08:00)
[User 1]: Get Motorcyclist #0 (08:00)
[User 1]: Status (08:00): Green
[User 1]: Status (08:00): Green
[User 1]: Amount (08:00): 1
[User 1]: Amount (08:00): 1
[User 1]: Status (08:30): White
[User 1]: Status (08:30): White
[User 1]: Amount (08:30): 2
[User 1]: Amount (08:30): 2
[User 2]: Status (08:00): White
[User 2]: Status (08:00): White
[User 2]: Amount (08:00): 1
[User 2]: Amount (08:00): 1
[User 2]: Status (08:30): White
[User 2]: Status (08:30): White
[User 2]: Amount (08:30): 2
[User 2]: Amount (08:30): 2
[User 2]: Get Motorcyclist #1 (08:00)
[User 2]: Get Motorcyclist #1 (08:00)
[User 2]: Status (08:00): Green
[User 2]: Status (08:00): Green
[User 